In [1]:
import polars as pl
import altair as alt
import matplotlib.pyplot as plt

In [2]:
data = pl.read_csv("./datasets/results/pop_compare.csv")
data

FileNotFoundError: No such file or directory (os error 2): ./datasets/results/pop_compare.csv

In [5]:
data.pivot("DISTRICT", index=)

SyntaxError: invalid syntax (1362943457.py, line 1)

In [9]:
origin = pl.read_excel("./datasets/original/poi_original.xlsx)
origin["Population"]

SyntaxError: unterminated string literal (detected at line 1) (2621049779.py, line 1)

In [ ]:
data = data.with_columns(((pl.col("Official Estimate") - pl.col("Dataset Estimate"))*100/pl.col("Official Estimate")).alias("percent off"))

In [ ]:
# Create a bar chart comparing Dataset Estimate and Official Estimate
plt.figure(figsize=(10, 6))
cities = data['DISTRICT']

# Plotting the estimates
plt.bar(cities, data['Dataset Estimate'], width=0.4, label='Dataset Estimate', align='center')
plt.bar(cities, data['Official Estimate'], width=0.4, label='Official Estimate', align='edge')

# Adding labels and title
plt.xlabel('District')
plt.ylabel('Population Estimate')
plt.title('Comparison of Dataset Estimate(Last updated 18 February 2023) vs. Official Estimate(02/2021)')
plt.xticks(rotation=90)
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
data.select(pl.col("percent off")).mean()

In [ ]:
full = pl.read_excel("./datasets/temp/around_poi_with_population.xlsx")
full

In [ ]:
import folium


m = folium.Map(location=(21.022672, 105.831461), zoom_start=14)

poi_id = 105
poi = full.filter(pl.col("poi_id").eq(poi_id)).to_dicts()

folium.Marker(
        location=[float(poi[0]["lat"]), float(poi[0]["lon"])],
        tooltip="Click me!",
        popup="",
        icon=folium.Icon(color="green"),
    ).add_to(m)

for place in poi:
    folium.Marker(
        location=[float(place["latitude"]), float(place["longitude"])],
        tooltip=place["title"],
        icon=folium.Icon(color="red"),
    ).add_to(m)
m

In [ ]:
origin

In [7]:
full.join(origin, left_on='poi_id', right_on='Unique Identifier')["poi_id", "population", "Population"]

NameError: name 'full' is not defined

In [5]:
data

COUNTRY,CITY,DISTRICT,TYPE,Dataset Estimate,Official Estimate
str,str,str,str,f64,f64
"""Vietnam""","""Hà Nội""","""Đống Đa""","""Quận""",478.032,401.7
"""Vietnam""","""Hà Nội""","""Đông Anh""","""Huyện""",427.819,374.9
"""Vietnam""","""Hà Nội""","""Hoàng Mai""","""Quận""",424.121,364.9
"""Vietnam""","""Hà Nội""","""Hai Bà Trưng""","""Quận""",369.003,315.9
"""Vietnam""","""Hà Nội""","""Chương Mỹ""","""Huyện""",364.014,309.6
…,…,…,…,…,…
"""Vietnam""","""Hà Nội""","""Thanh Oai""","""Huyện""",196.813,185.4
"""Vietnam""","""Hà Nội""","""Hoàn Kiếm""","""Quận""",182.718,155.9
"""Vietnam""","""Hà Nội""","""Đan Phượng""","""Huyện""",181.365,154.3


In [6]:
dataset_chart = alt.Chart(data).mark_bar().encode(
    x='DISTRICT:N',
    y='Dataset Estimate:Q'
)
official_chart = alt.Chart(data).mark_tick(color='red').encode(
    x='DISTRICT:N',
    y='Official Estimate:Q',
)
dataset_chart | official_chart

alt.HConcatChart(...)

In [3]:
df = pl.read_parquet("./datasets/raw/area_pois.parquet")
df.unique(subset=["id"])

id,lat,lon,name,categories
str,f64,f64,str,str
"""ChIJ0Xgy9vSsNTERWJmn68I1SbM""",20.983828,105.825164,"""Cafe Hoàng Hà""","""cafe,food,point_of_interest,es…"
"""ChIJRzX7UblTNDERjot3Bz5_zLA""",20.946902,105.755447,"""Coffee Nails""","""cafe,food,point_of_interest,es…"
"""ChIJnQbAyXStNTERQrpI-u__exo""",20.955802,105.846761,"""Trà đá Hải - Hiển (ngon quên l…","""cafe,store,food,point_of_inter…"
"""ChIJ441Zt5itNTERIx3M5El1Os0""",20.936135,105.850568,"""ATM Techcombank (CDM)""","""atm,finance,point_of_interest,…"
"""ChIJVa3hAABVNDERh36oEZ4wnRk""",21.02901,105.765769,"""Sports Coffee""","""cafe,food,point_of_interest,st…"
…,…,…,…,…
"""ChIJr-uKTXesNTERQhk5BfZ3-H4""",21.002853,105.841548,"""Văn phòng lấy mẫu MEDLATEC Đồn…","""hospital,point_of_interest,hea…"
"""ChIJX8NV8YKsNTERZ3soUVXjhBE""",21.008684,105.819341,"""Bệnh Viện Châm Cứu Trung Ương""","""hospital,health,point_of_inter…"
"""ChIJFwHFcFOtNTERdk0x86scTK0""",20.983928,105.846133,"""Techcombank Trương Định""","""bank,finance,point_of_interest…"


In [4]:
import folium


m = folium.Map(location=(21.022672, 105.831461), zoom_start=14)

places = df.unique(subset=["id"]).sample(n=100).to_dicts()

for place in places:
    folium.Marker(
        location=[float(place["lat"]), float(place["lon"])],
        tooltip=place["name"],
        icon=folium.Icon(color="red"),
    ).add_to(m)
m